In [1]:
import json
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 1. Load and preprocess AIS data
def load_ais_data(file_path):
    df = pd.read_csv(file_path, header=None, names=['id', 'timestamp', 'data'])
    df['data'] = df['data'].apply(json.loads)
    data_df = pd.json_normalize(df['data'])
    df = pd.concat([df.drop('data', axis=1), data_df], axis=1)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

ais_data = load_ais_data('rhein.csv')

# 2. Compute velocities and gradient per MMSI
ais_data['velocity_x'] = ais_data.groupby('mmsi')['lon'].diff() / ais_data.groupby('mmsi')['timestamp'].diff().dt.total_seconds()
ais_data['velocity_y'] = ais_data.groupby('mmsi')['lat'].diff() / ais_data.groupby('mmsi')['timestamp'].diff().dt.total_seconds()
ais_data['gradient_info'] = np.arctan2(ais_data['velocity_y'], ais_data['velocity_x'])

# 3. Voyage segmentation parameters
EPSILON = 1000        # max gap (s) within a voyage segment
MIN_DURATION = 1000   # minimum total duration (s) to keep a segment
RESAMPLE_FREQ = '60S' # resample to 1-minute intervals
SEQ_LEN = 10          # use 10 minutes as input

# 4. Split into continuous voyage segments per MMSI
voyage_segs = []
for mmsi, grp in ais_data.groupby('mmsi'):
    grp = grp.sort_values('timestamp').reset_index(drop=True)
    diffs = grp['timestamp'].diff().dt.total_seconds().fillna(0)
    idxs = [0]
    for i, dt in enumerate(diffs.iloc[1:], start=1):
        if dt < EPSILON:
            idxs.append(i)
        else:
            start, end = idxs[0], idxs[-1]
            if (grp.loc[end,'timestamp'] - grp.loc[start,'timestamp']).total_seconds() > MIN_DURATION:
                voyage_segs.append(grp.loc[idxs])
            idxs = [i]
    # last segment
    start, end = idxs[0], idxs[-1]
    if (grp.loc[end,'timestamp'] - grp.loc[start,'timestamp']).total_seconds() > MIN_DURATION:
        voyage_segs.append(grp.loc[idxs])

# 5. Interpolate each voyage to uniform 1-minute steps
uniform_voyages = []
for seg in voyage_segs:
    seg = seg.set_index('timestamp')
    t_index = pd.date_range(seg.index[0], seg.index[-1], freq=RESAMPLE_FREQ)
    seg_u = seg.reindex(t_index)
    for col in ['lat','lon','speed','turn']:
        seg_u[col] = seg_u[col].interpolate(method='cubic')
    seg_u = seg_u.dropna()
    uniform_voyages.append(seg_u[['lat','lon','speed','turn']])

# 6. Create N-to-1 sequences (input: 10×4, target: next lat/lon)
X_seqs, y_seqs = [], []
for vog in uniform_voyages:
    arr = vog.values
    for i in range(len(arr) - SEQ_LEN):
        X_seqs.append(arr[i:i+SEQ_LEN])
        y_seqs.append(arr[i+SEQ_LEN, :2])

X = np.stack(X_seqs)  # shape: (num_samples, 10, 4)
y = np.stack(y_seqs)  # shape: (num_samples, 2)

# 7. Normalize inputs and targets
scaler_X = MinMaxScaler()
ns, nt, nf = X.shape
X_flat = X.reshape(-1, nf)
X_scaled = scaler_X.fit_transform(X_flat).reshape(ns, nt, nf)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

# 8. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42
)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test  = torch.tensor(y_test,  dtype=torch.float32)

# 9. Define the LSTM model with bivariate Gaussian output
class VesselLSTM(nn.Module):
    def __init__(self, in_dim=4, emb_dim=128, hid_dim=128, layers=2):
        super().__init__()
        self.embed_in  = nn.Sequential(nn.Linear(in_dim, emb_dim), nn.Sigmoid())
        self.lstm      = nn.LSTM(emb_dim, hid_dim, layers, batch_first=True)
        self.embed_out = nn.Sequential(nn.Linear(hid_dim, hid_dim), nn.Sigmoid())
        self.param     = nn.Linear(hid_dim, 5)  # mu_x, mu_y, log_sx, log_sy, rho_raw

    def forward(self, x):
        B, T, _ = x.shape
        xin = self.embed_in(x.view(B*T, -1)).view(B, T, -1)
        lstm_out, _ = self.lstm(xin)
        h = self.embed_out(lstm_out[:, -1, :])
        p = self.param(h)
        mu_x, mu_y = p[:,0], p[:,1]
        sx, sy = torch.exp(p[:,2]), torch.exp(p[:,3])
        rho = torch.tanh(p[:,4])
        return mu_x, mu_y, sx, sy, rho

def bivar_nll(mu_x, mu_y, sx, sy, rho, y_true):
    x_t, y_t = y_true[:,0], y_true[:,1]
    eps = 1e-6
    nx = (x_t - mu_x)/(sx+eps); ny = (y_t - mu_y)/(sy+eps)
    z = nx**2 + ny**2 - 2*rho*nx*ny
    denom = 2*(1 - rho**2 + eps)
    log_norm = torch.log(2*np.pi*sx*sy*torch.sqrt(1 - rho**2 + eps))
    return (log_norm + z/denom).mean()

# 10. Train the model
model = VesselLSTM()
opt = Adam(model.parameters(), lr=0.001, weight_decay=0.5)
sched = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.95)
loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(X_train, y_train),
    batch_size=1024, shuffle=True
)

for epoch in range(100):
    model.train()
    total_loss = 0
    for xb, yb in loader:
        opt.zero_grad()
        mux, muy, sx, sy, rho = model(xb)
        loss = bivar_nll(mux, muy, sx, sy, rho, yb)
        loss.backward(); opt.step()
        total_loss += loss.item()*xb.size(0)
    sched.step()
    print(f"Epoch {epoch+1}/100, Loss: {total_loss/len(loader.dataset):.6f}")

# 11. Evaluate
model.eval()
with torch.no_grad():
    mux, muy, sx, sy, rho = model(X_test)
    test_nll = bivar_nll(mux, muy, sx, sy, rho, y_test)
print("Test NLL:", test_nll.item())



/var/folders/cb/xm1f2cfx06ggr6tw6gt2tnd40000gn/T/ipykernel_90343/1487851833.py:56: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  t_index = pd.date_range(seg.index[0], seg.index[-1], freq=RESAMPLE_FREQ)
/var/folders/cb/xm1f2cfx06ggr6tw6gt2tnd40000gn/T/ipykernel_90343/1487851833.py:56: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  t_index = pd.date_range(seg.index[0], seg.index[-1], freq=RESAMPLE_FREQ)
/var/folders/cb/xm1f2cfx06ggr6tw6gt2tnd40000gn/T/ipykernel_90343/1487851833.py:56: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  t_index = pd.date_range(seg.index[0], seg.index[-1], freq=RESAMPLE_FREQ)
/var/folders/cb/xm1f2cfx06ggr6tw6gt2tnd40000gn/T/ipykernel_90343/1487851833.py:56: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  t_index = pd.date_range(seg.index[0], seg.index[-

ValueError: The number of derivatives at boundaries does not match: expected 3, got 0+0